In [67]:
import numpy as np
from tqdm .auto import tqdm
import glob, os
from IPython.display import display as ipd
import IPython

In [75]:
true_path =  '/home/hemant/asr_wm/data/ner/txt/without_space/dev/*.txt'
pred_path = '/home/hemant/asr_wm/data/save_2/without_space/without_lm/beam/dev/*.txt'

stop = [']']

In [76]:
def rea(file):
    with open(file, 'r') as f:
        dummy = f.read()
    l = ''
    w = ''
    for i in dummy:
        if i in start:
            l+= i
            w= '-'
        elif w == '-' and len(l) != 0:
            l+= i
        if i in stop:
            l+= ','
            w = ''
    return l[:-1].split(',')

In [77]:
def main(true, pred):
    t = []
    for i in true:
        for j in i:
            if j: t.append(j)

    t_ = []
    for i in t:
        s,st =0, 0
        for j in i:
            if j in start:
                s+=1
            if j in stop:
                st+=1
        if s ==st:
            t_.append(i)    
    t = t_

    p = []
    for i in pred:
        for j in i:
            if j: p.append(j)

    p_ = []
    for i in p:
        s,st =0, 0
        for j in i:
            if j in start:
                s+=1
            if j in stop:
                st+=1
        if s ==st:
            p_.append(i)    
    p = p_
    return t,p

In [78]:
def score(tp, fp, fn):
    if tp == 0:
        pre, rec, f1 = 0, 0, 0
    else:
        pre = np.round(tp/(tp+fp),decimals=3)
        rec = np.round(tp/(tp+fn),decimals=3)
        f1 = np.round(2*((pre*rec)/(pre+rec)),decimals=3)
    return pre, rec, f1

In [79]:
true_ = glob.glob(true_path)
pred_ = glob.glob(pred_path)
prf = []
tpfp = []
assert len(true_) == len(pred_)
for j in ["|", "{", "$"]:
    start = [j]
    print(start)
    tp, fp, fn = 0, 0, 0
    t_ = []
    
    for i in tqdm(range(len(true_))):
        
        true = [rea(true_[i])]
        pred = [rea(pred_[i])]
        t, p = main(true,pred)
        t = [i for i in t]
        p = [i for i in p]

        tp+=  len(set(p).intersection(t))
        fp+= len(set(p) - set(t)) 
        fn+=  len(set(t) - set(p))

    pre, rec,f1 = score(tp, fp, fn)
    prf.append([pre,rec,f1])
    tpfp.append([tp,fp,fn])
    print(f"prec is {pre}\n recall is {rec}\n f1 score is {f1}")

['|']



prec is 0.355
 recall is 0.21
 f1 score is 0.264
['{']



prec is 0.13
 recall is 0.026
 f1 score is 0.043
['$']



prec is 0.487
 recall is 0.273
 f1 score is 0.35


In [80]:
#macro
np.round((prf[0][0] + prf[1][0] + prf[0][0] ) /3,3),np.round((prf[0][1] + prf[1][1] + prf[2][1] ) /3,3), np.round((prf[0][2] + prf[1][2] + prf[2][2] ) /3,3) 

(0.28, 0.17, 0.219)

In [81]:
#micro
score(tpfp[0][0] + tpfp[1][0] + tpfp[2][0], tpfp[0][1] +tpfp[1][1] + tpfp[2][1], tpfp[0][2] + tpfp[1][2] + tpfp[2][2])

(0.393, 0.219, 0.281)

In [66]:
start = ["|", "{", "$"]
for i in tqdm(range(len(true_))):
        
    true = [rea(true_[i])]
    pred = [rea(pred_[i])]
    t, p = main(true,pred)
    t = [i for i in t]
    p = [i for i in p]

    tp+=  len(set(p).intersection(t))
    fp+= len(set(p) - set(t)) 
    fn+=  len(set(t) - set(p))

pre, rec,f1 = score(tp, fp, fn)
print(f"prec is {pre}\n recall is {rec}\n f1 score is {f1}")


prec is 0.419
 recall is 0.223
 f1 score is 0.291


In [147]:
def rea_(file):
    with open(file, 'r') as f:
        return f.read()

In [ ]:
file_name = pred[2]
IPython.display.Audio('/home/hemant/asr_wm/data/ner/align_wav_txt/'
                      + os.path.basename(file_name).split('.')[0] + '.wav')

In [ ]:
for i in range(len(pred)):
    try:
        p,r,f1 = main([pred[i]],[true[i]],show=False)
        
        if p < 1:
            print('true: ',rea(true[i]))
            print('pred: ', rea(pred[i]))
            output = 'prec\trec\tf1\n'
            output +=f'{p}\t{r}\t{f1}\n'
            print(output)
            ipd(IPython.display.Audio('/home/hemant/asr_wm/data/ner/align_wav_txt/'+
                                      os.path.basename(pred[i]).split('.')[0] + '.wav'))
            print('--',pred[i])
    except: 
        print('skipped---',pred[i])
    print('--------------------------------------------------------')

In [ ]:
out =rea('/home/hemant/asr_wm/data/ner/pp_without_misc/without_space/out.txt').split('\n')

In [ ]:
for i in out:
    print(i,'\n----------------')

In [ ]:
for i in rea_('/home/hemant/asr_wm/data/ner/pp_without_misc/without_space/ner.csv'):
    command = f"cp {i.split(',')[-1].strip()} /home/hemant/dummy/{i.split(',')[-1].strip().split('/')[-1]}"
    os.system(command)